In [355]:
import pickle
import pandas as pd

pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)

with open(f"data/model_data/dev.pkl", 'rb') as fid:
    df = pickle.load(fid)
df = df[df.majority_vote > -1]
    
with open(f"data/model_data/dev-model_best-predictions.txt", 'r') as fid:
    preds = fid.readlines()
df['hard_label_preds'] = [int(x.strip()) for x in preds]

label_fns = [
    'isa_pattern', 
    'suchas_pattern',
    'including_pattern',
    'called_pattern',
    'especially_pattern',
    'appo_pattern', 
    'other_pattern', 
    'knownas_pattern', 
    'term_pos', 
    'term_modifier',
    'term_subset'
]
df.label_model_labels = [tuple(x) for x in df.label_model_labels]
df['label_fn_label'] = df.apply(lambda row: max([row[c] for c in label_fns]), axis=1)

In [359]:
df[df.hard_label_preds > 0] \
    .drop(
    ['tokens', 'chapter', 'textbook', 'sentence_id', 'term1', 'term2', 
     'term1_location', 'term2_location', 'label_model_labels'], axis=1) \
    .sort_values(['text', 'term_pair'])[['text', 'term_pair', 'gold_label', 'kb_bio101_ds_pos', 'hard_label_preds']] \
    .to_csv('manual_eval2.csv', index=False)

In [329]:
import pickle
import pandas as pd

pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)

with open(f"data/model_data/dev.pkl", 'rb') as fid:
    df = pickle.load(fid)
df = df[df.majority_vote > -1]
    
with open(f"data/model_data/dev-model_best-predictions.txt", 'r') as fid:
    preds = fid.readlines()
df['hard_label_preds'] = [int(x.strip()) for x in preds]

label_fns = [
    'isa_pattern', 
    'suchas_pattern',
    'including_pattern',
    'called_pattern',
    'especially_pattern',
    'appo_pattern', 
    'other_pattern', 
    'knownas_pattern', 
    'term_pos', 
    'term_modifier',
    'term_subset'
]
df.label_model_labels = [tuple(x) for x in df.label_model_labels]
df['label_fn_label'] = df.apply(lambda row: max([row[c] for c in label_fns]), axis=1)

In [330]:
def flip_tps(x, label):
    if x[label] == 1:
        return x['term_pair']
    else:
        return (x['term_pair'][1], x['term_pair'][0])
    
label_tps = df[df.gold_label > 0][['term_pair', 'gold_label']]
label_tps = set(label_tps.apply(lambda x: flip_tps(x, 'gold_label'), axis=1))
pred_tps = df[df.hard_label_preds > 0][['term_pair', 'hard_label_preds']]
pred_tps = set(pred_tps.apply(lambda x: flip_tps(x, 'hard_label_preds'), axis=1))

In [331]:
fp = pred_tps - label_tps 
tp = pred_tps.intersection(label_tps)
fn = label_tps - pred_tps

In [345]:
fp

{('-pron-', 'fibrous protein'),
 ('actin filament', 'microfilament'),
 ('animal cell', 'centrosome'),
 ('antibiotic resistance', 'trait'),
 ('archaea', 'prokaryote'),
 ('bacteria', 'prokaryote'),
 ('blood type', 'trait'),
 ('blood type', 'variation'),
 ('cardiac muscle', 'cell'),
 ('cdk', 'cyclin dependent kinase'),
 ('chromatin', 'complex'),
 ('dimer', 'globular protein'),
 ('egg', 'human cell'),
 ('gamete', 'human cell'),
 ('golgi body', 'golgi apparatus'),
 ('inner membrane', 'phospholipid bilayer'),
 ('intermediate filament', 'actin filament'),
 ('microvillus', 'plasma membrane'),
 ('multinucleate cell', 'nucleus'),
 ('myosin', 'motor protein'),
 ('nucleoid', 'cytoplasm'),
 ('nucleoid', 'genetic material'),
 ('plasma membrane', 'genetic material'),
 ('plasma membrane', 'phospholipid bilayer'),
 ('polar microtubule', 'chromosome'),
 ('protein hormone', 'protein'),
 ('proto oncogene', 'regulator'),
 ('sex cell', 'human cell'),
 ('thylakoid', 'granum'),
 ('xylem', 'vascular tissue')}

In [354]:
fp_rev = set([(x[1], x[0]) for x in fp])
df[(df.term_pair.isin(fp_rev))]

,textbook,chapter,sentence_id,text,tokens,term1,term1_location,term2,term2_location,term_pair,isa_pattern,suchas_pattern,including_pattern,called_pattern,especially_pattern,appo_pattern,other_pattern,knownas_pattern,term_pos,term_modifier,term_subset,kb_bio101_ds_pos,kb_bio101_ds_neg,majority_vote,label_model_labels,gold_label,hard_label_preds,label_fn_label
494,OpenStax Biology 2e,4,4.22,The plasma membranes of cells that specialize in absorption fold into fingerlike projections that we call microvilli (singular = microvillus); ( Figure 4.10 ).,"[The, plasma, membranes, of, cells, that, specialize, in, absorption, fold, into, fingerlike, projections, that, we, call, microvilli, (, singular, =, microvillus, ), ;, (, Figure, 4.10, ), .]",plasma membrane,"(1, 3)",microvillus,"(20, 21)","(plasma membrane, microvillus)",-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,0,"(0.4704208745427781, 0.26481429420036345, 0.2647648312568585)",0,2,-1
603,OpenStax Biology 2e,4,4.44,It consists of two phospholipid bilayers: an outer and an inner membrane.,"[It, consists, of, two, phospholipid, bilayers, :, an, outer, and, an, inner, membrane, .]",phospholipid bilayer,"(4, 6)",inner membrane,"(11, 13)","(phospholipid bilayer, inner membrane)",-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,0,"(0.4704208745427781, 0.26481429420036345, 0.2647648312568585)",0,2,-1
670,OpenStax Biology 2e,4,4.58,We call these unwound protein-chromosome complexes chromatin ( Figure 4.12 ).,"[We, call, these, unwound, protein, -, chromosome, complexes, , chromatin, , (, Figure, 4.12, ), .]",complex,"(7, 8)",chromatin,"(9, 10)","(complex, chromatin)",-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,0,"(0.4704208745427781, 0.26481429420036345, 0.2647648312568585)",0,0,-1
1066,OpenStax Biology 2e,4,4.113,The Centrosome The centrosome is a microtubule-organizing center found near the nuclei of animal cells.,"[The, Centrosome, , The, , centrosome, , is, a, microtubule, -, organizing, center, found, near, the, nuclei, of, animal, cells, .]",centrosome,"(5, 6)",animal cell,"(18, 20)","(centrosome, animal cell)",-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,0,"(0.4704208745427781, 0.26481429420036345, 0.2647648312568585)",0,0,-1
1464,OpenStax Biology 2e,4,5.21,"The proteins can also secrete from the cell (such as protein hormones, enzymes).","[The, proteins, can, also, secrete, from, the, cell, (, such, as, protein, hormones, ,, enzymes, ), .]",protein,"(1, 2)",protein hormone,"(11, 13)","(protein, protein hormone)",-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,0,"(0.4704208745427781, 0.26481429420036345, 0.2647648312568585)",0,2,-1
1534,OpenStax Biology 2e,4,5.45,"Sorting, tagging, packaging, and distributing lipids and proteins takes place in the Golgi apparatus (also called the Golgi body), a series of flattened membranes ( Figure 4.20 ).","[Sorting, ,, tagging, ,, packaging, ,, and, distributing, lipids, and, proteins, takes, place, in, the, , Golgi, apparatus, , (, also, called, the, Golgi, body, ), ,, a, series, of, flattened, membranes, (, Figure, 4.20, ), .]",golgi apparatus,"(16, 18)",golgi body,"(23, 25)","(golgi apparatus, golgi body)",-1,-1,-1,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,"(0.0007278648234553209, 0.49601695044447264, 0.5032551847320721)",0,2,0
1737,OpenStax Biology 2e,4,6.15,"ATP powers actin to assemble its filamentous form, which serves as a track for the movement of a motor protein we call myosin.","[ATP, powers, actin, to, assemble, its, filamentous, form, ,, which, serves, as, a, track, for, the, movement, of, a, motor, protein, we, call, myosin, .]",motor protein,"(19, 21)",myosin,"(23, 24)","(motor protein, myosin)",-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,0,"(0.4704208745427781, 0.26481429420036345, 0.2647648312568585)",0,2,-1
1987,OpenStax Biology 2e,4,7.23,Such transport uses the vascular tissues (xylem and phloem) primarily.,"[Such, transport, uses, the, vascular, tissues, (, xylem, and, phloem, ), primarily, .]",vascular tissue,"(4, 6)",xylem,"(7, 8)","(vascular tissue, xylem)",-1,-1,-1,-1,-1,-1,-1,

In [270]:
df[df.text == "In the active , dephosphorylated state, Rb binds to proteins called transcription factors , most commonly, E2F"
  ] 

,textbook,chapter,sentence_id,text,tokens,term1,term1_location,term2,term2_location,term_pair,isa_pattern,suchas_pattern,including_pattern,called_pattern,especially_pattern,appo_pattern,other_pattern,knownas_pattern,term_pos,term_modifier,term_subset,kb_bio101_ds_pos,kb_bio101_ds_neg,majority_vote,label_model_labels,hard_label_preds,label_fn_label


In [337]:
len(tp) / (len(tp) + len(fn))

0.3220338983050847

In [160]:
with open('false_positives.txt', 'w') as fid:
    fid.write('\n'.join([str(t) for t in fp]))
with open('true_positives.txt', 'w') as fid:
    fid.write('\n'.join([str(t) for t in tp]))
with open('false_negatives.txt', 'w') as fid:
    fid.write('\n'.join([str(t) for t in fn]))

In [200]:
(len(tp)) / (len(fp) + len(tp))

0.2619047619047619

In [199]:
(len(tp)) / (len(fn) + len(tp))

0.29931972789115646

In [58]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score 

true = df['kb_bio101_ds'].replace(-1, 0)
precision_score(true, df['label_fn_label'].astype(int), average='binary')
recall_score(true, df['label_fn_label'].astype(int), average='binary')
#f1_score(true, df['label_fn_label'].astype(int), average='binary')

0.24198250728862974

In [277]:
df[df.text == 'The condensed chromosomes are viewed within the nucleus (top), removed from a cell during mitosis (also called karyokinesis or nuclear division) and spread out on a slide (right), and artificially arranged according to length (left); an arrangement like this is called a karyotype.']

,textbook,chapter,sentence_id,text,tokens,term1,term1_location,term2,term2_location,term_pair,isa_pattern,suchas_pattern,including_pattern,called_pattern,especially_pattern,appo_pattern,other_pattern,knownas_pattern,term_pos,term_modifier,term_subset,kb_bio101_ds_pos,kb_bio101_ds_neg,majority_vote,label_model_labels,hard_label_preds,label_fn_label
17115,OpenStax Biology 2e,10,2.23,"The condensed chromosomes are viewed within the nucleus (top), removed from a cell during mitosis (also called karyokinesis or nuclear division) and spread out on a slide (right), and artificially arranged according to length (left); an arrangement like this is called a karyotype.","[The, condensed, chromosomes, are, viewed, within, the, nucleus, (, top, ), ,, removed, from, a, cell, during, mitosis, (, also, called, karyokinesis, or, nuclear, division, ), and, spread, out, on, a, slide, (, right, ), ,, and, artificially, arranged, according, to, length, (, left, ), ;, an, arrangement, like, this, is, called, a, karyotype, .]",condensed chromosome,"(1, 3)",nuclear division,"(23, 25)","(condensed chromosome, nuclear division)",-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,0,"(0.4812712333618878, 0.259393114468755, 0.2593356521693572)",0,-1
17116,OpenStax Biology 2e,10,2.23,"The condensed chromosomes are viewed within the nucleus (top), removed from a cell during mitosis (also called karyokinesis or nuclear division) and spread out on a slide (right), and artificially arranged according to length (left); an arrangement like this is called a karyotype.","[The, condensed, chromosomes, are, viewed, within, the, nucleus, (, top, ), ,, removed, from, a, cell, during, mitosis, (, also, called, karyokinesis, or, nuclear, division, ), and, spread, out, on, a, slide, (, right, ), ,, and, artificially, arranged, according, to, length, (, left, ), ;, an, arrangement, like, this, is, called, a, karyotype, .]",nucleus,"(7, 8)",nuclear division,"(23, 25)","(nucleus, nuclear division)",-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,0,"(0.4812712333618878, 0.259393114468755, 0.2593356521693572)",0,-1
17117,OpenStax Biology 2e,10,2.23,"The condensed chromosomes are viewed within the nucleus (top), removed from a cell during mitosis (also called karyokinesis or nuclear division) and spread out on a slide (right), and artificially arranged according to length (left); an arrangement like this is called a karyotype.","[The, condensed, chromosomes, are, viewed, within, the, nucleus, (, top, ), ,, removed, from, a, cell, during, mitosis, (, also, called, karyokinesis, or, nuclear, division, ), and, spread, out, on, a, slide, (, right, ), ,, and, artificially, arranged, according, to, length, (, left, ), ;, an, arrangement, like, this, is, called, a, karyotype, .]",mitosis,"(17, 18)",nuclear division,"(23, 25)","(mitosis, nuclear division)",-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,1,"(3.591366290125828e-05, 0.644172680619265, 0.35579140571783363)",0,-1
17118,OpenStax Biology 2e,10,2.23,"The condensed chromosomes are viewed within the nucleus (top), removed from a cell during mitosis (also called karyokinesis or nuclear division) and spread out on a slide (right), and artificially arranged according to length (left); an arrangement like this is called a karyotype.","[The, condensed, chromosomes, are, viewed, within, the, nucleus, (, top, ), ,, removed, from, a, cell, during, mitosis, (, also, called, karyokinesis, or, nuclear, division, ), and, spread, out, on, a, slide, (, right, ), ,, and, artificially, arranged, according, to, length, (, left, ), ;, an, arrangement, like, this, is, called, a, karyotype, .]",nuclear division,"(23, 25)",leave,"(43, 44)","(nuclear division, leave)",-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,0,"(0.4812712333618878, 0.259393114468755, 0.2593356521693572)",0,-1
17120,OpenStax Biology 2e,10,2.23,"The condensed chromosomes are viewed within the nucleus (top), removed from a cell during mitosis (also called karyokinesis or nuclear division) and spread out on a slide (right